In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [1]:
%%writefile vector.cu
#include<bits/stdc++.h>
#include<cstdlib>
using namespace std;
#define N 512

__global__ void add(int *a,int *b,int *c){
  int idx = blockDim.x * blockIdx.x + threadIdx.x;
  if(idx<N)
  {
    c[idx] = a[idx] + b[idx];
  }
}

int main(){
  int *a,*b,*c;
  int *da,*db,*dc;
  int size = N*sizeof(int);

  a = (int *)malloc(size);
  b = (int *)malloc(size);
  c = (int *)malloc(size);

  cudaMalloc((void **)&da,size);
  cudaMalloc((void **)&db,size);
  cudaMalloc((void **)&dc,size);

  for(int i=0;i<N;i++){
    a[i]=i;
    b[i]=i*2;
  }

  cudaMemcpy(da,a,size,cudaMemcpyHostToDevice);
  cudaMemcpy(db,b,size,cudaMemcpyHostToDevice);
  cudaMemcpy(dc,c,size,cudaMemcpyHostToDevice);

  add<<<N,1>>>(da,db,dc);

  cudaMemcpy(c,dc,size,cudaMemcpyDeviceToHost);

  cout <<"c[0] = "<<c[0]<<" ,c["<<N-1<<"] = "<<c[N-1];
  free(a);
  free(b);
  free(c);

  cudaFree(da);
  cudaFree(db);
  cudaFree(dc);

  return 0;

}

Writing vector.cu


In [2]:
!nvcc vector.cu -o vect

In [3]:
!./vect


c[0] = 0 ,c[511] = 1533

In [4]:
!nvprof ./vect

==488== NVPROF is profiling process 488, command: ./vect
==488== Profiling application: ./vect
c[0] = 0 ,c[511] = 1533==488== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   44.42%  5.6000us         1  5.6000us  5.6000us  5.6000us  add(int*, int*, int*)
                   38.07%  4.8000us         3  1.6000us  1.5040us  1.7920us  [CUDA memcpy HtoD]
                   17.51%  2.2080us         1  2.2080us  2.2080us  2.2080us  [CUDA memcpy DtoH]
      API calls:   98.97%  126.62ms         3  42.208ms  2.6800us  126.62ms  cudaMalloc
                    0.76%  975.75us         1  975.75us  975.75us  975.75us  cuDeviceGetPCIBusId
                    0.11%  140.83us       101  1.3940us     133ns  62.183us  cuDeviceGetAttribute
                    0.07%  89.367us         3  29.789us  2.7770us  80.410us  cudaFree
                    0.05%  62.075us         4  15.518us  8.0630us  23.428us  cudaMemcpy
                    0.02% 